## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-20-53-02 +0000,nypost,Don Lemon’s lawsuit against Elon Musk for axin...,https://nypost.com/2025/07/10/media/don-lemons...
1,2025-07-10-20-47-56 +0000,nyt,"Coal, the Last Survivor of Canada’s Parliament...",https://www.nytimes.com/2025/07/10/world/canad...
2,2025-07-10-20-34-33 +0000,nyt,Ukrainian Intelligence Officer Is Shot to Deat...,https://www.nytimes.com/2025/07/10/world/europ...
3,2025-07-10-20-28-31 +0000,nypost,White House financial watchdog rips Jerome Pow...,https://nypost.com/2025/07/10/business/white-h...
4,2025-07-10-20-22-01 +0000,nyt,US-Brazil Tariffs: What to Know About Trump’s ...,https://www.nytimes.com/2025/07/10/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2353/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,18
39,brazil,7
40,tariffs,7
78,will,7
175,gaza,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
38,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,49
61,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,46
89,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,43
24,2025-07-10-18-19-52 +0000,bbc,Why is Trump targeting Brazil - and will it ba...,https://www.bbc.com/news/articles/crk68drj57mo,39
91,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,39


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
38,49,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
62,23,2025-07-10-12-00-00 +0000,nypost,I was fed up with ‘toxic’ sunscreens so I star...,https://nypost.com/2025/07/10/health/my-nontox...
33,22,2025-07-10-16-43-05 +0000,bbc,UK faces rising and unpredictable threat from ...,https://www.bbc.com/news/articles/c98wjzj4jlpo
91,21,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo
0,20,2025-07-10-20-53-02 +0000,nypost,Don Lemon’s lawsuit against Elon Musk for axin...,https://nypost.com/2025/07/10/media/don-lemons...
69,20,2025-07-10-10-50-45 +0000,bbc,Deadly new Russian drone and missile attack hi...,https://www.bbc.com/news/articles/cj3rvpe06rxo
92,19,2025-07-10-00-36-34 +0000,bbc,"US sanctions UN expert Francesca Albanese, cri...",https://www.bbc.com/news/articles/c70rllxr0kyo
39,17,2025-07-10-15-43-16 +0000,nypost,Former Moët Hennessy exec claims ‘sexist’ boss...,https://nypost.com/2025/07/10/business/former-...
25,16,2025-07-10-18-00-31 +0000,bbc,"'One in, one out' sounds simple - but the migr...",https://www.bbc.com/news/articles/c8d60djgqndo
37,15,2025-07-10-16-08-22 +0000,bbc,Russia's intensifying drone war is spreading f...,https://www.bbc.com/news/articles/c0m8gn7grn2o


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
